In [5]:
import pandas as pd
import os
from openpyxl import load_workbook
import numpy as np
import re

In [2]:
geo=pd.read_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\szószok\\api_ideig.xlsx")
late=pd.read_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\df\\dfveg_latest_2.xlsx")

geo.drop(columns='Unnamed: 0', inplace=True)
late.drop(columns=['Unnamed: 0', 0, 2, 4, 5, 7, 8], inplace=True)
incidents=pd.concat([late, geo], axis=1)
incidents.head()

,3,6,9,ship type 2,otket,lat1,long,cloud_cover_low,temperature_2m,wind_speed_10m,wind_speed_100m,weather_code,wind_gusts_10m,rain
0,Attempted,0900 hrs,"North of lrian Maya, Indonesia",&GENERAL CARGO SHIP,2010-01-03,1.656667,132.755000,10.0,28.144499,15.893093,17.026896,51.0,22.319998,0.2
1,CAT 3,0200 hrs,"Koja Continer Terminal, Tanjung Priok, Indonesia",&CONTAINER SHIP,2010-01-04,-6.097667,106.904167,2.0,28.593500,19.813087,24.162947,0.0,30.599998,0.0
2,CAT 3,0400 hrs,"Petrokimia Terminal, Gresik Port, Indonesia",&CHEMICAL TANKER,2010-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CAT 3,0205 hrs,"Year Vung Tau Anchorage, Vietnam",&CONTAINER SHIP,2010-01-12,10.250000,107.006667,10.0,27.019501,14.417988,18.416384,3.0,34.200001,0.0
4,CAT 3,0415 hrs,"Near North Pulau Laut Coal Terminal, Indonesia",&BULK CARRIER,2010-01-13,-1.356667,116.988333,1.0,28.722000,8.496305,9.085988,3.0,15.480000,0.0


In [4]:
#szűrés SOMS-ra, majd havi alapra hozás
df=incidents
df1 = df[df[9].str.contains(r"(?=.*Malacca)(?=.*Singapore)|SOMS|S0MS", na=False)]
df1.index=pd.to_datetime(df1.otket)
df1.index
df1.to_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\szószok\\inc_monthly_jo.xlsx")
mon= df1.resample('M').size().reset_index(name='incidents_per_month')
mon.index=pd.to_datetime(mon.otket)
def one(df):
    df.index = df.index.map(lambda x: x.replace(day=1))
    return df

mon=one(mon)
mon.drop(columns="otket", inplace=True)
mon.head()
mon.to_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\szószok\\Indonesia\\veg\\inc_monthly.xlsx")

In [6]:
#merging az országos datával
df_ind=pd.read_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\szószok\\Indonesia\\veg\\ind_final.xlsx")
df_mal=pd.read_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\szószok\\Malaysia\\mal_final.xlsx")
df_sg=pd.read_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\szószok\\Singapore (idk, kell-e)\\sg_final.xlsx")

ind=[]
mal=[]
sg=[]
for x in df_ind.columns:
    ind.append("Indonesia: "+str(x))

for x in df_mal.columns:
    mal.append("Malaysia: "+str(x))

for x in df_sg.columns:
    sg.append("Singapore: "+str(x))

df_mal.columns=mal
df_sg.columns=sg
df_ind.columns=ind

df_mal.index=pd.to_datetime(df_mal["Malaysia: Unnamed: 0"])
df_mal.drop(columns="Malaysia: Unnamed: 0", inplace=True)
df_ind.head()
df_ind.head()
df_ind.index=pd.to_datetime(df_ind['Indonesia: Unnamed: 0'])
df_ind.drop(columns="Indonesia: Unnamed: 0", inplace=True)

df_sg.head()
df_sg.index=pd.to_datetime(df_sg['Singapore: Unnamed: 0'])
df_sg.drop(columns="Singapore: Unnamed: 0", inplace=True)


monthlydata=pd.concat([df_mal, df_sg, df_ind, mon], axis=1)


In [7]:
#lag
#először is rászűrni 2008+-ra
m2=monthlydata.iloc[648:829]
df=m2
from08 = [col for col in df.columns if df[col].first_valid_index() is not None and df[col].first_valid_index() == pd.Timestamp('2008-01-01')]




In [8]:
df2=pd.DataFrame(df)
for x in from08:
    df[f'{x}_lag24'] = df[x].shift(24)





C:\Users\Andris\AppData\Local\Temp\ipykernel_26280\1047966451.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{x}_lag24'] = df[x].shift(24)
C:\Users\Andris\AppData\Local\Temp\ipykernel_26280\1047966451.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{x}_lag24'] = df[x].shift(24)
C:\Users\Andris\AppData\Local\Temp\ipykernel_26280\1047966451.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [49]:
df2

,Malaysia: FPCPITOTLZGMYS (linear),Malaysia: FPCPITOTLZGMYS (time),Malaysia: uncertainty (linear),Malaysia: uncertainty (time),"Malaysia: Population, total","Malaysia: GNI, Atlas method (current US$)",Malaysia: GDP (current US$),"Malaysia: Inflation, GDP deflator (annual %)","Malaysia: Agriculture, forestry, and fishing, value added (% of GDP)","Malaysia: Population, total.1",...,Indonesia: Linear interpolation: Adjusted net national income (current US$)_lag24,Indonesia: Linear interpolation: Adjusted net national income per capita (current US$)_lag24,Indonesia: real_gdp_ind_seas_adj (linear)_lag24,Indonesia: real_gdp_ind_seas_adj (time)_lag24,Indonesia: nominal_gdp_ind_seas_adj (linear)_lag24,Indonesia: nominal_gdp_ind_seas_adj (time)_lag24,Indonesia: real_gdp_ind_non_seas_adj (linear)_lag24,Indonesia: real_gdp_ind_non_seas_adj (time)_lag24,Indonesia: nominal_gdp_ind_non_seas_adj (linear)_lag24,Indonesia: nominal_gdp_ind_non_seas_adj (time)_lag24
2008-01-01,5.440782,5.440782,0.098600,0.098600,2.714024e+07,1.793546e+11,1.966547e+11,5.358333,10.000000,2.714103e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-02-01,5.035993,5.055900,0.100215,0.100161,2.718789e+07,1.816530e+11,1.997599e+11,5.816667,10.000000,2.718632e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-03-01,4.631203,4.644475,0.101830,0.101830,2.723553e+07,1.839514e+11,2.028651e+11,6.275000,10.000000,2.723475e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-04-01,4.226414,4.246321,0.103445,0.103445,2.728317e+07,1.862498e+11,2.059703e+11,6.733333,10.000000,2.728161e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-01,3.821624,3.834896,0.232426,0.233828,2.733081e+07,1.885483e+11,2.090754e+11,7.191667,10.000000,2.733003e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,2.785477,2.788727,0.210126,0.209631,3.384713e+07,3.912154e+11,3.987287e+11,6.225000,9.075000,3.384639e+07,...,8.145058e+11,3002.586053,2.699819e+06,2.699604e+06,3896346.00,3.895934e+06,2.713308e+06,2.713347e+06,3.920225e+06,3.920103e+06
2022-10-01,2.711324,2.713152,0.255715,0.255715,3.387750e+07,3.946305e+11,4.014949e+11,6.283333,9.016667,3.387733e+07,...,8.117301e+11,2990.204182,2.719584e+06,2.719584e+06,3934243.25,3.934243e+06,2.709722e+06,2.709722e+06,3.931411e+06,3.931411e+06
2022-11-01,2.637171,2.640015,0.205561,0.206651,3.390786e+07,3.980457e+11,4.042612e+11,6.341667,8.958333,3.390728e+07,...,8.089543e+11,2977.822311,2.729246e+06,2.729036e+06,3977394.75,3.976457e+06,2.701296e+06,2.701479e+06,3.945252e+06,3.944951e+06
2022-12-01,2.563018,2.564440,0.155406,0.155952,3.393822e+07,4.014608e+11,4.070275e+11,6.400000,8.900000,3.393822e+07,...,8.061786e+11,2965.440440,2.738907e+06,2.738802e+06,4020546.25,4.020077e+06,2.692871e+06,2.692962e+06,3.959092e+06,3.958942e+06


In [10]:
dfmon=df.iloc[24:]
df2=df2.iloc[24:]
df2.head()

,Malaysia: overall,Malaysia: agriculture,Malaysia: mining,Malaysia: manufacturing,Malaysia: electricity,Malaysia: water,Malaysia: lf,Malaysia: lf_employed,Malaysia: lf_unemployed,Malaysia: lf_outside,...,Indonesia: Crime in SULAWESI SELATAN (time),Indonesia: Crime in SULAWESI TENGGARA (time),Indonesia: Crime in GORONTALO (time),Indonesia: Crime in SULAWESI BARAT (time),Indonesia: Crime in MALUKU (time),Indonesia: Crime in MALUKU UTARA (time),Indonesia: Crime in PAPUA BARAT (time),Indonesia: Crime in PAPUA (time),Indonesia: Crime in INDONESIA (time),incidents_per_month
2010-01-01,98.4,92.7,99.4,98.8,99.8,99.2,12367.9,11931.2,436.7,6927.5,...,16870.186301,6134.690411,3845.912329,NaN,2691.791781,1179.369863,NaN,6039.926027,343884.432877,1.0
2010-02-01,98.4,93.3,97.8,99.0,99.4,99.1,12059.8,11632.3,427.4,7225.1,...,16779.128767,6139.830137,3781.704110,NaN,2801.797260,1241.123288,NaN,5960.375342,342929.210959,0.0
2010-03-01,99.9,96.1,108.6,99.2,100.5,99.2,12324.2,11895.9,428.3,6993.0,...,16678.315068,6145.520548,3710.616438,NaN,2923.589041,1309.493151,NaN,5872.301370,341871.643836,0.0
2010-04-01,99.8,95.3,107.7,99.3,99.7,99.9,12512.0,12133.5,378.5,6894.5,...,16580.753425,6151.027397,3641.821918,NaN,3041.452055,1375.657534,NaN,5787.068493,340848.191781,0.0
2010-05-01,98.6,94.1,91.4,99.8,100.3,101.5,12197.6,11798.9,398.7,7186.0,...,16479.939726,6156.717808,3570.734247,NaN,3163.243836,1444.027397,NaN,5698.994521,339790.624658,1.0


In [11]:
dfmon.to_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\monthly-data.xlsx")

dfmon

,Malaysia: FPCPITOTLZGMYS (linear),Malaysia: FPCPITOTLZGMYS (time),Malaysia: uncertainty (linear),Malaysia: uncertainty (time),"Malaysia: Population, total","Malaysia: GNI, Atlas method (current US$)",Malaysia: GDP (current US$),"Malaysia: Inflation, GDP deflator (annual %)","Malaysia: Agriculture, forestry, and fishing, value added (% of GDP)","Malaysia: Population, total.1",...,Indonesia: Linear interpolation: Adjusted net national income (current US$)_lag24,Indonesia: Linear interpolation: Adjusted net national income per capita (current US$)_lag24,Indonesia: real_gdp_ind_seas_adj (linear)_lag24,Indonesia: real_gdp_ind_seas_adj (time)_lag24,Indonesia: nominal_gdp_ind_seas_adj (linear)_lag24,Indonesia: nominal_gdp_ind_seas_adj (time)_lag24,Indonesia: real_gdp_ind_non_seas_adj (linear)_lag24,Indonesia: real_gdp_ind_non_seas_adj (time)_lag24,Indonesia: nominal_gdp_ind_non_seas_adj (linear)_lag24,Indonesia: nominal_gdp_ind_non_seas_adj (time)_lag24
2010-01-01,1.622852,1.622852,0.184997,0.184997,2.825891e+07,2.126832e+11,2.066541e+11,-4.891667,9.275000,2.825971e+07,...,3.141212e+11,1335.813896,1.513323e+06,1.513323e+06,1.259852e+06,1.259852e+06,1.475278e+06,1.475278e+06,1.207305e+06,1.207305e+06
2010-02-01,1.752154,1.741881,0.186439,0.186358,2.830063e+07,2.145252e+11,2.110508e+11,-3.783333,9.350000,2.829811e+07,...,3.188089e+11,1354.094994,1.520674e+06,1.520429e+06,1.282677e+06,1.281916e+06,1.495639e+06,1.494961e+06,1.249181e+06,1.247785e+06
2010-03-01,1.881455,1.873662,0.187880,0.187864,2.834234e+07,2.163672e+11,2.154475e+11,-2.675000,9.425000,2.834062e+07,...,3.234966e+11,1372.376091,1.528026e+06,1.528026e+06,1.305502e+06,1.305502e+06,1.516001e+06,1.516001e+06,1.291057e+06,1.291057e+06
2010-04-01,2.010757,2.001192,0.189322,0.189322,2.838405e+07,2.182092e+11,2.198442e+11,-1.566667,9.500000,2.838176e+07,...,3.281844e+11,1390.657189,1.535378e+06,1.535378e+06,1.328328e+06,1.328328e+06,1.536362e+06,1.536362e+06,1.332933e+06,1.332933e+06
2010-05-01,2.140059,2.132974,0.126215,0.125529,2.842576e+07,2.200511e+11,2.242409e+11,-0.458333,9.575000,2.842427e+07,...,3.328721e+11,1408.938287,1.542168e+06,1.542242e+06,1.351049e+06,1.351296e+06,1.559463e+06,1.559714e+06,1.375603e+06,1.376067e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,2.785477,2.788727,0.210126,0.209631,3.384713e+07,3.912154e+11,3.987287e+11,6.225000,9.075000,3.384639e+07,...,8.145058e+11,3002.586053,2.699819e+06,2.699604e+06,3.896346e+06,3.895934e+06,2.713308e+06,2.713347e+06,3.920225e+06,3.920103e+06
2022-10-01,2.711324,2.713152,0.255715,0.255715,3.387750e+07,3.946305e+11,4.014949e+11,6.283333,9.016667,3.387733e+07,...,8.117301e+11,2990.204182,2.719584e+06,2.719584e+06,3.934243e+06,3.934243e+06,2.709722e+06,2.709722e+06,3.931411e+06,3.931411e+06
2022-11-01,2.637171,2.640015,0.205561,0.206651,3.390786e+07,3.980457e+11,4.042612e+11,6.341667,8.958333,3.390728e+07,...,8.089543e+11,2977.822311,2.729246e+06,2.729036e+06,3.977395e+06,3.976457e+06,2.701296e+06,2.701479e+06,3.945252e+06,3.944951e+06
2022-12-01,2.563018,2.564440,0.155406,0.155952,3.393822e+07,4.014608e+11,4.070275e+11,6.400000,8.900000,3.393822e+07,...,8.061786e+11,2965.440440,2.738907e+06,2.738802e+06,4.020546e+06,4.020077e+06,2.692871e+06,2.692962e+06,3.959092e+06,3.958942e+06


In [13]:
from08 = [col for col in df2.columns if df2[col].first_valid_index() is not None and df[col].first_valid_index() == pd.Timestamp('2008-01-01')]
for x in from08:
    df2.drop(columns=x, inplace=True)

df2.to_excel("C:\\Users\\Andris\\Documents\\Szakdoga\\monthly-data_only_lagged.xlsx")
